In [ ]:
import subprocess
import shutil
import os
from pdf2image import convert_from_path
import math
from PIL import Image
import moviepy.video.io.ImageSequenceClip
from git import Repo

## 1. Specifying the paths

In [37]:
filepath = '/Users/gromit/topoXML/vgtc_journal_latex/'
gitpath = '/Users/gromit/topoXML/'
main_tex = 'main'
total_duration=52 # duration of the video
n_cols = 6 # number of pages in one row
n_rows = 2 # number of rows

## 2. Fetch all the commits in master branch

In [ ]:
current_path = os.path.abspath(os.getcwd())

if not os.path.exists('intermediate'):
    os.mkdir('intermediate')
else:
    shutil.rmtree('{}/intermediate'.format(current_path))
    os.mkdir('intermediate')
    
repo = Repo(gitpath)

commits = repo.iter_commits('master')

commit_ids = []
for commit in commits:
    commit_ids.append(commit.hexsha)
commit_ids = commit_ids[::-1]

## 3. Generating Images per Commits - May take a long time!

In [ ]:
for order,c in enumerate(commit_ids):
    os.chdir(gitpath)
    subprocess.check_call(['git', 'checkout', c])
    
    os.chdir(filepath)
    try:
        subprocess.check_call(['pdflatex', '{}.tex'.format(main_tex)])
        subprocess.check_call(['bibtex', '{}'.format(main_tex)])
        subprocess.check_call(['pdflatex', '{}.tex'.format(main_tex)])
        subprocess.check_call(['pdflatex', '{}.tex'.format(main_tex)])
    except:
        continue

    try:
        shutil.move('{}{}.pdf'.format(filepath,main_tex),current_path)
    except:
        os.remove('{}/{}.pdf'.format(current_path,main_tex))
        shutil.move('{}{}.pdf'.format(filepath,main_tex),current_path)
    
    images = convert_from_path('{}/{}.pdf'.format(current_path,main_tex))

    width, height = images[0].size

    stiched_image = Image.new("RGB", (n_cols*width, height*n_rows))

    for i, img in enumerate(images): 
        x = (i % n_cols) * width
        if i >= n_cols:
            y = height
        else:
            y = 0
        stiched_image.paste(img, (x , y, x + width , y + height))

    width, height = stiched_image.size
    width = int(width/2)
    height = int(height/2)
    stiched_image = stiched_image.resize((width,height), Image.ANTIALIAS)
    stiched_image.save('{}/intermediate/{}.png'.format(current_path,order), 'PNG')

## 4. Make the video from the images, done!

In [38]:
image_folder='{}/intermediate'.format(current_path)

image_files = [img for img in os.listdir(image_folder) if img.endswith(".png")]
image_files.sort(key=lambda f: int(f.split('.')[0]))
image_files = ['{}/{}'.format(image_folder,img) for img in image_files]
clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=len(image_files) / total_duration)
clip.write_videofile('{}/video.mp4'.format(current_path))

t:   0%|          | 0/62 [00:00<?, ?it/s, now=None]

Moviepy - Building video /Users/gromit/paper_stitch/video.mp4.
Moviepy - Writing video /Users/gromit/paper_stitch/video.mp4



Moviepy - Done !
Moviepy - video ready /Users/gromit/paper_stitch/video.mp4
